In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [ ]:
priceData = pd.read_csv("./data/processed/pricesList.csv")
rainfallData = pd.read_csv("./data/processed/rainfallData.csv")

In [ ]:
# extract only one vegetable for the prediction
priceData = priceData[priceData["Name"] == "Potatoes_Local_POTATOES_1Kg"]

In [ ]:
def featureEngineering(data):
    data2 = data.copy()
    # iterate through the rainfallData and add the rainfall to the priceData
    # for index, row in rainfallData.iterrows():
    #     # match the year and month from the rainfall data to the pricedata
    #     year = row['year']
    #     month = row['month']
    #     data2.loc[(data2['Year'] == year) & (data2['Month'] == month), 'anuradhapura'] = row['anuradhapura']
    #     data2.loc[(data2['Year'] == year) & (data2['Month'] == month), 'jaffna'] = row['jaffna']
    #     data2.loc[(data2['Year'] == year) & (data2['Month'] == month), 'nuwaraeliya'] = row['nuwaraeliya']

    return data2
    

In [ ]:
def extractDate(data):
    data2 = data.copy()
    data2["Datetime"] = pd.to_datetime(data2[['Year', 'Month']].assign(day=(data2["Week"]-1)*7 +1))
    data2 = data2.set_index('Datetime')
    return data2

In [ ]:
def preprocessData(data):
    data2 = data.copy()
    data2 = data2.dropna()
    return data2

In [ ]:
# Removing outliers
from scipy.stats import gaussian_kde
def remove_outliers_kde(df, column_name, threshold=0.05):
    # Extract the values of the column
    column_values = df[column_name].values
    
    # Fit the kernel density estimation
    kde = gaussian_kde(column_values)
    
    # Evaluate the KDE for each data point
    density = kde.evaluate(column_values)
    
    # Sort the data points by their density values
    sorted_indices = np.argsort(density)
    
    # Calculate the threshold index based on the given threshold
    threshold_index = int(len(sorted_indices) * threshold)
    
    # Get the indices of non-outliers
    non_outlier_indices = sorted_indices[threshold_index:]
    
    # Filter out the non-outliers
    df_cleaned = df.iloc[non_outlier_indices]
    df_cleaned = df_cleaned.reset_index(drop=True)
    return df_cleaned

In [ ]:
df = featureEngineering(priceData);
df = preprocessData(df);
# df = remove_outliers_kde(df, 'Price')
df = extractDate(df)

In [ ]:
df["Price"].plot(style='.',figsize=(15, 5), title="Price Fluxuation of Carrot in Sri Lanka")
plt.show()

In [ ]:
df.info()

In [ ]:
split_date = "2022-08-01"
train = df.loc[df.index < split_date]
test = df.loc[df.index >= split_date]

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(15, 5))
train['Price'].plot(ax=ax, label="Train Set", style=".")
test['Price'].plot(ax=ax, label="Test Set", style=".")
ax.axvline(split_date, color="black", ls="--")
ax.legend("Training set", "Test Set")
plt.show()

In [ ]:
# plot week of data
df["Price"].loc[(df.index > "2019-01-01") & (df.index < "2019-05-01")].plot(style=".", figsize=(15, 5), title="Price Fluxuation of Carrot in Sri Lanka")

In [ ]:
df.columns

In [ ]:
FEATURES = ["Week", "Month", "Year", 
            # "anuradhapura", 
            # "jaffna", 
            # "nuwaraeliya"
            ]
TARGET = "Price"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]


In [ ]:
# Model Creation
reg = xgb.XGBRegressor(n_estimators=10000, learning_rate=0.001)
reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=2500,verbose=50)
# reg.fit(X_test, y_test)

In [ ]:
# evaluate the model
y_pred = reg.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse}")
# score
score = reg.score(X_test, y_test)
print(f"Score: {score}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(bootstrap=True, criterion='poisson',max_depth=None, max_leaf_nodes=None,n_estimators=10000, random_state=None, n_jobs=1, verbose=0)
reg.fit(X_train, y_train)
# reg.fit(X_test, y_test)

In [ ]:
from catboost import CatBoostRegressor

reg = CatBoostRegressor(iterations=70000, depth=5, learning_rate=0.001, loss_function='RMSE')
reg.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)
# reg.fit(X_test, y_test)

In [ ]:
from prophet import Prophet
model = Prophet()
model.fit(train.reset_index().rename(columns={"Datetime": "ds", "Price": "y"}))
forecast = model.predict(test.reset_index().rename(columns={"Datetime": "ds", "Price": "y"}))

In [ ]:
fix, ax = plt.subplots(figsize=(10, 5))
fig = model.plot(forecast, ax=ax)
plt.show()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.scatter(test.index, test["Price"], color="r", label="True Price")
fig = model.plot(forecast, ax=ax)
plt.show()


In [ ]:
# forecast on test
style = "-"
test2 = test.copy()
test2["prediction"] = reg.predict(X_test)
df2 = df.merge(test2[["prediction"]], how="left", left_index=True, right_index=True)
ax = df2[["Price"]].plot(style=style, figsize=(15,5))
df2["prediction"].plot(style=style, ax=ax)

In [ ]:
ax = df2.loc[(df2.index > "2022-07-01") & (df2.index < "2025-01-01")][["Price"]].plot(figsize=(15,5), style=style)
df2.loc[(df2.index > "2022-07-01") & (df2.index < "2025-01-01")]["prediction"].plot(ax=ax, style=style)

In [ ]:
# model refit
X_train_copy = X_train.copy()
X_train_copy = pd.concat([X_train_copy, X_test.loc[X_test.index < "2023-12-01"].copy()])
y_train_copy = y_train.copy()
y_train_copy = pd.concat([y_train_copy, y_test.loc[y_test.index < "2023-12-01"].copy()])
reg.fit(X_train_copy, y_train_copy)

# Arima

In [ ]:
from pmdarima import auto_arima

In [ ]:
df2 = df.copy()
stepwise_fit = auto_arima(df2["Price"], trace=True, suppress_warnings=True)
stepwise_fit.summary()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
model = ARIMA(train["Price"], order=(2,1,2))
model = model.fit()
model.summary()

In [ ]:
start = len(train)
end = len(train) + len(test) - 1
predictions = model.predict(start=start, end=end, typ="levels").rename("ARIMA Predictions")
predictions.index = test.index
print(predictions)

In [ ]:
predictions.plot(legend=True)
test["Price"].plot(legend=True)

# Time series cross validation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
tss = TimeSeriesSplit(n_splits=5, test_size=1, gap=0)
df3 = df.copy().sort_index()

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(15, 10))
fold = 0
for train_index, test_index in tss.split(df3):
    train = df3.iloc[train_index]
    test = df3.iloc[test_index]
    train["Price"].plot(ax=axs[fold], label="Train")
    test["Price"].plot(ax=axs[fold], label="Test")
    axs[fold].axvline(test.index.min(), color="black", ls="--")
    fold += 1